In [1]:
import dask
import dask.dataframe as dd
from dask_ml.datasets import make_regression
from dask_ml.model_selection import train_test_split
from dask_ml.preprocessing import MinMaxScaler, QuantileTransformer,RobustScaler, StandardScaler
#import dask_xgboost as dxgb
from dask.distributed import Client
import xgboost as xgb


# Connect to Dask cluster
#client = Client()
# check null and sum


In [2]:
f ='/media/ljp238/6tb/Joseph/DATASETS/OUT_TILES/1x1degree/N10_E105/patch256x256/outproc/Train1001patches/tile_256_256_idx295.parquet'

In [28]:
data = dd.read_parquet(f)

In [29]:
data.head()

,x,y,zdif,lidar,tdx,cop,merit,nasa,aw3d,s2r,s2g,s2b,id
0,105.028444,10.971556,-3.402823e+38,3.402823e+38,2.505653,2.424999,3.677196,0.837923,5.253370,0.122705,0.092074,0.062667,295
1,105.028556,10.971556,-3.402823e+38,3.402823e+38,2.208611,2.350650,3.672680,0.508425,5.075640,0.125213,0.096221,0.064349,295
2,105.028668,10.971556,-3.402823e+38,3.402823e+38,2.522614,2.399151,3.668847,0.242785,4.915590,0.122470,0.103785,0.068503,295
3,105.028779,10.971556,-3.402823e+38,3.402823e+38,2.917932,2.517056,3.665547,0.134711,4.822927,0.115115,0.102685,0.070169,295
4,105.028891,10.971556,-3.402823e+38,3.402823e+38,2.586941,2.585482,3.662641,0.280167,4.781820,0.105068,0.096805,0.066856,295


In [41]:
dat = data.loc[data["zdif"] > -99]#.compute()
dat.compute()

,x,y,zdif,lidar,tdx,cop,merit,nasa,aw3d,s2r,s2g,s2b,id


In [42]:
dat.head()

,x,y,zdif,lidar,tdx,cop,merit,nasa,aw3d,s2r,s2g,s2b,id


In [5]:
#data.isna().sum()

Dask Series Structure:
npartitions=1
aw3d    int64
zdif      ...
dtype: int64
Dask Name: dataframe-sum-agg, 4 graph layers

In [6]:
rcv_cols = ['zdif', 'cop', 'merit', 'nasa', 'aw3d', 's2r','s2g', 's2b']

In [7]:
df = data[rcv_cols]

In [8]:
#df.describe()

In [9]:
X = df.drop('zdif', axis=1)
y = df['zdif']#
X.shape

(Delayed('int-43b2d69f-9e66-48b5-963b-dbd3f9c21d53'), 7)

In [10]:
trainx, testx, trainy, testy = train_test_split(X, y, shuffle=True,test_size=.15)
#print(f'{trainx.shape} {} {} {}')

In [11]:
X.shape

(Delayed('int-7ffd4677-c0f8-48f1-8d1a-9fed0081db47'), 7)

In [12]:
# use pipelines instead

In [13]:
#from dask_ml.xgboost import XGBRegressor

In [14]:
if __name__ == "__main__":
    cluster = dask.distributed.LocalCluster()
    client = dask.distributed.Client(cluster)

    dtrain = xgb.dask.DaskDMatrix(client, X, y)

    output = xgb.dask.train(
        client,
        {"verbosity": 2, "tree_method": "hist", "objective": "reg:squarederror"},
        dtrain,
        num_boost_round=4,
        evals=[(dtrain, "train")],
    )



INFO:distributed.scheduler:Receive client connection: Client-worker-57d066dc-c23b-11ed-8bea-dc4a3e70c0fa
INFO:distributed.core:Starting established connection to tcp://127.0.0.1:49458
[07:39:25] task [xgboost.dask-7]:tcp://127.0.0.1:32955 got new rank 0
[07:39:25] INFO: ../src/gbm/gbtree.cc:179: Tree method is selected to be 'hist', which uses a single updater grow_quantile_histmaker.
[07:39:25] INFO: ../src/gbm/gbtree.cc:179: Tree method is selected to be 'hist', which uses a single updater grow_quantile_histmaker.
[07:39:25] INFO: ../src/data/simple_dmatrix.cc:102: Generating new Gradient Index.


[0]	train-rmse:inf
[1]	train-rmse:inf
[2]	train-rmse:inf
[3]	train-rmse:inf


[07:39:25] INFO: ../src/gbm/gbtree.cc:179: Tree method is selected to be 'hist', which uses a single updater grow_quantile_histmaker.
[07:39:25] INFO: ../src/gbm/gbtree.cc:179: Tree method is selected to be 'hist', which uses a single updater grow_quantile_histmaker.
[07:39:25] INFO: ../src/gbm/gbtree.cc:179: Tree method is selected to be 'hist', which uses a single updater grow_quantile_histmaker.
